In [1]:
import django_initializer
from api.models.user.user import User
from django.contrib.auth.models import User as AuthUser
from json import loads, dumps
import json
import lib
import re
import service
from api.models.campaign.campaign import Campaign
from api.models.user.user_subscription import UserSubscription
from service.web_crawler.facebook_shared_list_crawler import FacebookSharedListCrawler
import pickle
import platform
import time
from typing import OrderedDict
import functools, logging, traceback
from api import models
import service
from django.conf import settings
import os
import random
import lib
import arrow
from selenium import webdriver
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver import ActionChains, Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.color import Color
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from service.web_crawler.facebook_crawler import FacebookCrawler
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
def to_dict(input_ordered_dict):
    return loads(dumps(input_ordered_dict))

In [9]:
campaign_lucky_draw_id = "1505"
lucky_draw = lib.util.verify.Verify.get_lucky_draw(campaign_lucky_draw_id)
username = lucky_draw.campaign.facebook_page.username
post_id = lucky_draw.campaign.facebook_campaign.get("post_id", "")
fb_crawler = FacebookSharedListCrawler(username, post_id, open_browser=True)
shared_user_name_set = fb_crawler.start()
print(shared_user_name_set)

remote
login with cookies
validate_user
check if Go to App button shows up
Traceback (most recent call last):
  File "/home/liveshowseller/service/web_crawler/facebook_shared_list_crawler.py", line 73, in validate_user
    go_to_app_button = self.driver.find_element(By.CSS_SELECTOR, "button[value='Go to App']")
  File "/root/.cache/pypoetry/virtualenvs/liveshowseller-4MbNXGbY-py3.9/lib/python3.9/site-packages/selenium/webdriver/remote/webdriver.py", line 856, in find_element
    return self.execute(Command.FIND_ELEMENT, {
  File "/root/.cache/pypoetry/virtualenvs/liveshowseller-4MbNXGbY-py3.9/lib/python3.9/site-packages/selenium/webdriver/remote/webdriver.py", line 429, in execute
    self.error_handler.check_response(response)
  File "/root/.cache/pypoetry/virtualenvs/liveshowseller-4MbNXGbY-py3.9/lib/python3.9/site-packages/selenium/webdriver/remote/errorhandler.py", line 243, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.NoSuchEl

In [2]:
class RenewFacebookCookiesCrawler(FacebookCrawler):
    def __init__(self, open_browser=False, chromedriver_path=None):
        super().__init__(open_browser, chromedriver_path)
    def login(self, validate=False):
        print("login")
        count = 0
        while not self.is_login and count <= 4:
            if not validate:
                self.driver.get(self.login_url)
            email_input = self.driver.find_element(By.ID, "m_login_email")
            pass_input = self.driver.find_element(By.ID, "m_login_password")
            login_button = self.driver.find_element(By.NAME, "login")
            self.actions.send_keys_to_element(email_input, self.email)
            self.actions.send_keys_to_element(pass_input, self.password)
            self.actions.click(login_button)
            self.actions.perform()
            count += 1
            self.is_login = True
        time.sleep(2)
        self.validate_user()
        self.save_cookies()
        
        
    def validate_user(self):
        print("validate_user")
        # check if page pops up log in with one tap
        try:
            self.driver.find_element(By.CSS_SELECTOR, "input[value='regular_login']")
            ok_button = self.driver.find_element(By.CSS_SELECTOR, "button[value='OK']")
            self.actions.click(ok_button).perform()
            self.save_login = True
        except:
            print(traceback.format_exc())
            
        if not self.save_login:
            # check new page identity double check
            try:
                my_profile = self.driver.find_element(By.CSS_SELECTOR, f"div[role='button']")
                self.actions.click(my_profile).perform()

                pass_input = self.wait.until(
                    EC.presence_of_element_located((By.NAME, "pass"))
                )
                self.actions.send_keys_to_element(pass_input, self.password)
                login_button = self.driver.find_element(By.CSS_SELECTOR , "button[type='submit']")
                self.actions.click(login_button).perform()
                self.save_login = True
            except:
                print(traceback.format_exc())
            
        if not self.save_login:
            # check header identity double check
            try:
                login_button = self.driver.find_element(By.CSS_SELECTOR , "button[type='submit']")
                self.actions.click(login_button).perform()
                pass_input = self.wait.until(
                    EC.presence_of_element_located((By.NAME, "pass"))
                )
                self.actions.send_keys_to_element(pass_input, self.password)
                login_button = self.driver.find_element(By.CSS_SELECTOR , "button[type='submit']")
                self.actions.click(login_button).perform()
                self.save_login = True
            except:
                print(traceback.format_exc())
    
    def start(self):
        try:
            self.create_driver()
            self.create_wait_object()
            self.create_action_object()
            self.login()
            self.driver.quit()
        except Exception as e:
            print(traceback.format_exc())
            if self.driver:
                self.driver.quit()

In [4]:
crawler = RenewFacebookCookiesCrawler(open_browser=False, chromedriver_path=os.path.join(settings.BASE_DIR, 'chromedriver'))
crawler.start()

remote
/home/liveshowseller/chromedriver
login
validate_user
Traceback (most recent call last):
  File "/tmp/ipykernel_22694/2593949759.py", line 28, in validate_user
    self.driver.find_element(By.CSS_SELECTOR, "input[value='regular_login']")
  File "/root/.cache/pypoetry/virtualenvs/liveshowseller-4MbNXGbY-py3.9/lib/python3.9/site-packages/selenium/webdriver/remote/webdriver.py", line 856, in find_element
    return self.execute(Command.FIND_ELEMENT, {
  File "/root/.cache/pypoetry/virtualenvs/liveshowseller-4MbNXGbY-py3.9/lib/python3.9/site-packages/selenium/webdriver/remote/webdriver.py", line 429, in execute
    self.error_handler.check_response(response)
  File "/root/.cache/pypoetry/virtualenvs/liveshowseller-4MbNXGbY-py3.9/lib/python3.9/site-packages/selenium/webdriver/remote/errorhandler.py", line 243, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.NoSuchElementException: Message: no such element: Unable to locate element: 

In [4]:
page_username = "kuai.mai.2021"
target_post_id = "517999680166258"
language = "zh_hant"
langauag_map = {
    "zh_hant": "中文(台灣)",
    "en": "English (US)",
    "zh_hans": "中文(简体)",
    "vi": "Tiếng Việt"
}
fb_crawler = FacebookSharedListCrawler(page_username, target_post_id, lang=language)